In [1]:
def label_img(img):
    train_id = img.split('_')[-3]
    side = img.split('_')[-2]
    frame = img.split('_')[-1]
    frame = frame.split('.')[-2]
    return [train_id, side, frame]

In [2]:
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm      # a nice pretty percentage bar for tasks. Thanks to viewer Daniel BA1/4hler for this suggestion

TRAIN_DIR = 'C:/Training/Train'
IMG_SIZE = 50

In [3]:
def wagon_end(img):
    queryImage = cv2.imread('C:/Training/tab.jpg',0)
    trainingImage = img

# create SIFT and detect/compute
    sift = cv2.xfeatures2d.SIFT_create()
    kp1, des1 = sift.detectAndCompute(queryImage,None)
    kp2, des2 = sift.detectAndCompute(trainingImage,None)

# FLANN matcher parameters
    FLANN_INDEX_KDTREE = 0
    indexParams = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    searchParams = dict(checks=50)   # or pass empty dictionary

    flann = cv2.FlannBasedMatcher(indexParams,searchParams)

    matches = flann.knnMatch(des1,des2,k=2)

# prepare an empty mask to draw good matches
    matchesMask = [[0,0] for i in range(len(matches))]

# David G. Lowe's ratio test, populate the mask
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.6*n.distance:
            return 1
    return 0

In [4]:
def create_train_data(TRAIN_DIR):
    training_data = []
           
    for dirName, subdirList, fileList in os.walk(TRAIN_DIR):
        wagon = w = 0
        for i in tqdm(range(len(fileList))):
            dirName = dirName.replace("\\","/")
            dirN = dirName.split('/')[-1]
            if len(dirN) > 4:
                img = dirN + '_' + str(i) + '.jpg'
                if img.endswith('.jpg'):
                    label = label_img(img)
                    path = os.path.join(dirName,img)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    crop = img[100:1000, 300:1000]
                    if w > 10:
                        if wagon_end(crop) == 1: 
                            wagon = wagon + 1
                            w = 0
                    w = w + 1
                    label.append(wagon)
                    crop = cv2.resize(crop, (IMG_SIZE,IMG_SIZE))
                    crop = crop.reshape(1,-1)
                    crop = np.float32(crop)
                    training_data.append([np.array(crop), np.array(label)])
                
    training_data = list(training_data)
    return training_data

data = create_train_data(TRAIN_DIR)

data = np.vstack(data)
print(data)

0it [00:00, ?it/s]
100%|████████████████████████████████████████| 193/193 [00:29<00:00,  6.46it/s]


[[array([[186., 184., 185., ..., 121., 110., 126.]], dtype=float32)
  array(['38', 'left', '0', '0'], dtype='<U4')]
 [array([[185., 184., 185., ..., 119., 109., 130.]], dtype=float32)
  array(['38', 'left', '1', '0'], dtype='<U4')]
 [array([[185., 183., 185., ..., 116., 107., 132.]], dtype=float32)
  array(['38', 'left', '2', '0'], dtype='<U4')]
 [array([[186., 183., 185., ...,  58.,  62.,  62.]], dtype=float32)
  array(['38', 'left', '3', '0'], dtype='<U4')]
 [array([[185., 184., 184., ...,  62.,  76.,  69.]], dtype=float32)
  array(['38', 'left', '4', '0'], dtype='<U4')]
 [array([[187., 186., 187., ...,  62.,  61.,  63.]], dtype=float32)
  array(['38', 'left', '5', '0'], dtype='<U4')]
 [array([[195., 194., 194., ...,  66.,  67.,  69.]], dtype=float32)
  array(['38', 'left', '6', '0'], dtype='<U4')]
 [array([[207., 209., 211., ...,  75.,  77.,  81.]], dtype=float32)
  array(['38', 'left', '7', '0'], dtype='<U4')]
 [array([[186., 187., 181., ...,  67.,  65.,  71.]], dtype=float32)
  ar

  array(['38', 'right', '192', '4'], dtype='<U5')]]
